In [1]:
from tensorflow.keras.callbacks import LearningRateScheduler, TensorBoard
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import warnings
import glob 
import os 
from eli5.sklearn import PermutationImportance
import eli5
from sklearn.metrics import make_scorer
import shap

# Make NumPy printouts easier to read.
np.set_printoptions(precision=5, suppress=True)
warnings.filterwarnings("ignore")

print(tf.__version__)

2.10.0


In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
DATASET_CALOSC=pd.read_csv('../Dane/pogoda/Obrobione_norm.csv')
DATASET_CALOSC=DATASET_CALOSC.drop(['przekaźnik pokojowy','praca_pieca','temperatura wewnętrzna na kominku','temperatura wewnętrzna nad kuchenką gazową'],axis=1)
DATASET_CALOSC

,temperatura zewnętrzna,wilgotność zewnętrzna,ciśnienie atmosferyczne,prędkość wiatru,kierunek wiatru,wilgotność wewnętrzna,temperatura wewnętrzna
0,0.00000,0.94856,0.96873,0.074708,0.22284,0.77525,0.93153
1,0.00000,0.94650,0.96873,0.108670,0.25627,0.77525,0.93153
2,0.00000,0.94650,0.96873,0.112060,0.25627,0.77005,0.93153
3,0.00000,0.94547,0.96873,0.112060,0.23398,0.77005,0.93153
4,0.00000,0.94753,0.96873,0.098478,0.27298,0.77005,0.93415
...,...,...,...,...,...,...,...
78093,0.77232,0.41328,0.99532,0.048325,0.59610,0.69846,0.91841
78094,0.77758,0.40623,0.99532,0.052765,0.84423,0.70116,0.91841
78095,0.77505,0.39643,0.99525,0.054333,0.51725,0.70241,0.91841
78096,0.76979,0.39708,0.99522,0.041272,0.45618,0.69846,0.91841


In [4]:
x = DATASET_CALOSC.iloc[:, :6]  
y = DATASET_CALOSC.iloc[:, 6]   

normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(x)

In [5]:
def custom_scheduler(epoch, lr):
    if epoch < 4:
        return lr
    else:
        return lr * tf.math.exp(-0.1)


model = tf.keras.models.Sequential([
    normalizer,
          tf.keras.layers.Dense(512, activation='relu'),
          tf.keras.layers.Dense(256, activation='relu'),
          tf.keras.layers.Dense(128, activation='relu'),
          tf.keras.layers.Dense(64, activation='relu'),
          tf.keras.layers.Dense(32, activation='relu'),
          tf.keras.layers.Dense(1,activation='linear') 
])

model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.001),
    loss="mse",
    metrics=['mae'])

scheduler = LearningRateScheduler(custom_scheduler)
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=1)
callbacks = [scheduler, tensorboard]

EPOCHS = 50
BATCH_SIZE=32

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
model.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=callbacks)

Epoch 1/50
1953/1953 [==============================] - 6s 3ms/step - loss: 0.0034 - mae: 0.0229 - lr: 0.0010
Epoch 2/50
1953/1953 [==============================] - 7s 4ms/step - loss: 2.1356e-04 - mae: 0.0113 - lr: 0.0010
Epoch 3/50
1953/1953 [==============================] - 5s 3ms/step - loss: 1.5988e-04 - mae: 0.0098 - lr: 0.0010
Epoch 4/50
1953/1953 [==============================] - 5s 3ms/step - loss: 1.3683e-04 - mae: 0.0090 - lr: 0.0010
Epoch 5/50
1953/1953 [==============================] - 5s 3ms/step - loss: 1.1425e-04 - mae: 0.0081 - lr: 9.0484e-04
Epoch 6/50
1953/1953 [==============================] - 5s 3ms/step - loss: 9.5807e-05 - mae: 0.0073 - lr: 8.1873e-04
Epoch 7/50
1953/1953 [==============================] - 5s 3ms/step - loss: 8.0311e-05 - mae: 0.0067 - lr: 7.4082e-04
Epoch 8/50
1953/1953 [==============================] - 5s 3ms/step - loss: 6.7677e-05 - mae: 0.0061 - lr: 6.7032e-04
Epoch 9/50
1953/1953 [==============================] - 5s 3ms/step - loss: 

In [7]:
model.evaluate(x_test, y_test)

489/489 [==============================] - 1s 2ms/step - loss: 1.5842e-05 - mae: 0.0025


[1.5842395441723056e-05, 0.0024539895821362734]

# Permutation Feature Importance

In [8]:
def model_score(x, y):
    return mean_squared_error(y, model.predict(x))

In [9]:
scorer = make_scorer(model_score, greater_is_better=True)

perm = PermutationImportance(model, scoring=scorer, random_state=1).fit(x_test, y_test)


489/489 [==============================] - 0s 961us/step


In [10]:
eli5.show_weights(perm, feature_names = x_test.columns.tolist())

Weight,Feature
0.0013 ± 0.0000,temperatura zewnętrzna
0.0012 ± 0.0000,wilgotność wewnętrzna
0.0012 ± 0.0000,ciśnienie atmosferyczne
0.0009 ± 0.0000,wilgotność zewnętrzna
0.0005 ± 0.0000,prędkość wiatru
0.0004 ± 0.0000,kierunek wiatru


# SHapley Additive exPlanations

In [11]:
# Przekształć dane do formatu NumPy, jeśli są w Pandas DataFrame
x_train_array = np.array(x_train)
x_test_array = np.array(x_test)

# Utwórz obiekt explainera SHAP
explainer = shap.DeepExplainer(model, x_train_array)

# Oblicz wartości SHAP dla testowego zestawu danych
shap_values = explainer.shap_values(x_test_array)

# # Wizualizacja
shap.summary_plot(shap_values, x_test_array)

TypeError: 'NoneType' object cannot be interpreted as an integer